    Author: Eswar Lakshminarayanan

In [47]:
import pandas as pd
import pickle
import datetime
import matplotlib.pyplot as plt

In [48]:
%%time
df = pd.read_csv("kiana_data.csv",header=0,nrows=1000000)

CPU times: user 2.27 s, sys: 93.5 ms, total: 2.36 s
Wall time: 1.17 s


### Are all timestamps are UTC ?

In [5]:
%%time
df[['just_date','just_time',"ignore"]] = df['localtime'].str.split(expand=True)

CPU times: user 2.46 s, sys: 157 ms, total: 2.62 s
Wall time: 2.62 s


In [37]:
df.ignore.unique()

array(['UTC'], dtype=object)

### Do visitors go to different levels in a visit?

In [43]:
%%time
import pandas as pd
chunk_df = pd.read_csv("kiana_data.csv",header=0, chunksize=10000)

CPU times: user 2.81 ms, sys: 9 µs, total: 2.81 ms
Wall time: 1.82 ms


#### The two functions below reshapes the data into a long trajectory format

In [ ]:
%%time
counter = 0
list_ = []
for d in chunk_df:
    if counter<10:
        counter+=1
        temp_d = d.groupby(["ClientMacAddr","just_date"]).apply(lambda grp: grp[["Level","lat","lng","just_time"]].values.tolist()).to_frame()
        temp_d = temp_d.reset_index()
        list_.append(temp_d)
    else:
        break
        
frame = pd.concat(list_, axis = 0, ignore_index = True)

In [8]:
frame.columns = ["mac_id","date","loc_info"]
frame.head(2)

,mac_id,date,loc_info
0,0,2018-07-11,"[[L5, 1.29086614, 103.85172392, 18:42:35.942],..."
1,0,2018-07-12,"[[L5, 1.29086614, 103.85172392, 19:12:05.708],..."


In [36]:
%%time
t2 = frame.groupby(["mac_id","date"]).agg({'loc_info': 'sum'}).reset_index()

CPU times: user 9.07 s, sys: 228 ms, total: 9.3 s
Wall time: 9 s


In [37]:
t2.head(5)

,mac_id,date,loc_info
0,8,2018-04-04,"[[L1, 1.2901341, 103.851295, 16:52:15]]"
1,11,2018-04-04,"[[L1, 1.2900982, 103.85135, 15:38:57]]"
2,36,2018-04-06,"[[L1, 1.2903618, 103.851746, 14:03:53]]"
3,36,2018-04-15,"[[L1, 1.2901032, 103.85134, 13:51:42], [L1, 1...."
4,36,2018-05-13,"[[L1, 1.2901355, 103.85129, 16:56:10]]"


### The function below checks if the trajectories have more than one levels (L1,LB1,L5) in them

In [57]:
def check(z_list):
    first_val = None
    if len(z_list)>1:
        for sub_l in z_list:
            if not first_val:
                first_val = sub_l[0]
            else:
                if first_val== sub_l[0]:
                    return False
                else:
                    return True
    else:
        return False

In [58]:
tmp = t2["loc_info"].apply(lambda z: check(z))

In [68]:
t2.loc[tmp].iloc[0,2]

[['L5', 1.2904733, 103.85147434, '02:48:18.336'],
 ['L1', 1.2896977862160839, 103.8511046543257, '23:54:48.226']]